In [30]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import requests
import json
import time
# this function helps me retriev the webpages 
def extract_content(url):
    executable_path=r"C:\Users\PC\Desktop\projects\Web_Scraper\chromedriver-win64\chromedriver-win64\chromedriver.exe"
    options = Options()
    service=Service(executable_path=executable_path)
    options.add_argument('--headless')  
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')

    # You can specify the path to chromedriver if it's not in PATH
    driver = webdriver.Chrome(service=service,options=options)

    driver.get(url)
    time.sleep(3)  # Wait for full page to load (can adjust)

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    driver.quit()

    # Remove unwanted elements
    for tag in soup(['script', 'style', 'nav', 'footer', 'header']):
        tag.decompose()

    text = soup.get_text(separator='\n', strip=True)
    return text




     

In [31]:
from duckduckgo_search import DDGS
#this function helps me fine the urls which mustly correspond to the question i ask
def search_url(question,results=3):
    urls=[]
    with DDGS() as ddgs:
        for r in ddgs.text(question,max_results=results):
            urls.append(r['href'])
    return urls

In [32]:
#web search for urls
question=input()
urls=search_url(question)
for i,url in enumerate(urls,0):
    print(f"{i},{url}")


 what documents a required to register at the tu ilmenau?


0,https://www.tu-ilmenau.de/en/study/in-study/course-of-studies/administrative-guide/application-documents
1,https://oia.ntut.edu.tw/var/file/32/1032/img/3207/S_InformationSheetHSP2021_2022.pdf
2,https://www.tu-ilmenau.de/


In [33]:
#with this prompt i try to summarize the content of the pages a retrieve using an llm
def summary_prompt(url_content):
    prompt = f"""You are a helpful assistant tasked with summarizing a web article.

Your goal is to provide a clear, concise, and accurate summary that captures all key points and nuances from the original text. Focus only on the relevant information that answers the user's query context.

Guidelines:
- Be faithful to the original meaning.
- Do not omit important details.
- Use your own words avoid copying long phrases.
- Keep the summary under approximately 1500 words.
- Ensure the summary is well-organized and easy to read.
-Provide plain text only.
-Answer in German only

Text to summarize:
{url_content}

Respond only with the final summary json below:
summary
"""
    return prompt

    
    

In [34]:
# i want to use i AI to generate a ground-Truth-answer base of the summary i provide to it
# what documents a required to register at the tu ilmenau?
def ground_truth_prompt(question,summary):
    prompt=f"""
    you are a human friendly assistant  ,and your role is to answer {question} base on the provided {summary}
Guidelines:
- Be faithful to the original meaning.
- Do not omit important details.
- Use your own words avoid copying long phrases.
- the length of your answer may be withen the range [10 ,300]
- give a well organised answer 
- Provide plain text only.
- the language you answer with depends on the language the {question} is asked.
give your answer hier:
    answer
    """
    return prompt
    
    

In [35]:
model="mistral"
#this function is responsible for the request and Response
def Query_llm(prompt: str, model: str, host: str = "http://localhost:11434"):
    url = f"{host}/api/generate"
    payload = {
        "model": model,
        "prompt": prompt,
        "stream": False  
    }

    try:
        response = requests.post(url, json=payload)
        response.raise_for_status()
        summary=response.json()
        return summary
    except requests.exceptions.RequestException as e:
        print("Ollama API error:", e)
        return ""

In [36]:
state={}
state["question"]=question
state["context"]={}

In [ ]:
question=state["question"]
print(question)
for i ,link in enumerate(urls):
    state["context"][f"link{i}"]=link
    print(state["context"][f"link{i}"])
    url_content=extract_content(link)
    summary_prompt=summary_prompt(url_content)
    summary=Query_llm(summary_prompt,model)
    ground_truth_prompt=ground_truth_prompt(question,summary)
    Answer=Query_llm(ground_truth_prompt,model)
    state["context"][f"Answer{i}"]= answer.get("response")
    print(state["context"][f"summary{i}"]["summary"])
    print(state["context"][f"Answer{i}"]["answer"])




what documents a required to register at the tu ilmenau?
https://www.tu-ilmenau.de/en/study/in-study/course-of-studies/administrative-guide/application-documents
